In [23]:
import pandas as pd

test = pd.read_csv('test.csv')
a = pd.read_csv('autogluon_stack3.csv')
a = a[['CI_HOUR']]
a = a.rename(columns={'CI_HOUR': 'CI_HOUR_NEW'})
test = test[['ARI_CO','ARI_PO','ATA','U_WIND', 'V_WIND','AIR_TEMPERATURE','BN']]
test = pd.concat([test,a],axis=1)

In [32]:
import pandas as pd

# 데이터 불러오기
#full_df = pd.read_csv("OLD-NEW.csv")
full_df = test.copy()
full_df['ATA'] = pd.to_datetime(full_df['ATA'])

# 매핑 딕셔너리 생성
'''
mapping_dict_global = {(row['ARI_PO'], row['U_WIND'], row['V_WIND'], row['AIR_TEMPERATURE'], row['BN']): (row['ATA'], row['CI_HOUR_NEW'])
                       for _, row in full_df.iterrows() if row['CI_HOUR_NEW'] <= 5}
'''
mapping_dict_global = {
    (row['ARI_PO'], row['U_WIND'], row['V_WIND'], row['AIR_TEMPERATURE'], row['BN']): (row['ATA'], row['CI_HOUR_NEW'])
    for _, row in full_df.iterrows() 
    if row['CI_HOUR_NEW'] <= 1 and not any(pd.isna([row['ARI_PO'], row['U_WIND'], row['V_WIND'], row['AIR_TEMPERATURE'], row['BN']]))
}

# 새로운 타겟 값 계산 함수
def compute_new_target_global(row):
    key = (row['ARI_PO'], row['U_WIND'], row['V_WIND'], row['AIR_TEMPERATURE'], row['BN'])
    if key in mapping_dict_global and mapping_dict_global[key][0] > row['ATA']:
        time_difference = (mapping_dict_global[key][0] - row['ATA']).total_seconds() / 3600
        return time_difference
    return None

# 새로운 타겟 값 계산
full_df['New_Target'] = full_df.apply(compute_new_target_global, axis=1)
import numpy as np
full_df['New_Target'] = np.where((full_df['New_Target'] > 2159) | full_df['New_Target'].isna(), np.nan, full_df['New_Target'])
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220491 entries, 0 to 220490
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   ARI_CO           220491 non-null  object        
 1   ARI_PO           220491 non-null  object        
 2   ATA              220491 non-null  datetime64[ns]
 3   U_WIND           128766 non-null  float64       
 4   V_WIND           128766 non-null  float64       
 5   AIR_TEMPERATURE  128245 non-null  float64       
 6   BN               128766 non-null  float64       
 7   CI_HOUR_NEW      220491 non-null  float64       
 8   New_Target       15418 non-null   float64       
dtypes: datetime64[ns](1), float64(6), object(2)
memory usage: 15.1+ MB


In [33]:
full_df['CI_HOUR_NEW'] = full_df['CI_HOUR_NEW'].where(full_df['New_Target'].isna(), full_df['New_Target'])
full_df

,ARI_CO,ARI_PO,ATA,U_WIND,V_WIND,AIR_TEMPERATURE,BN,CI_HOUR_NEW,New_Target
0,SG,GIW5,2020-06-18 11:58:00,0.37,1.63,27.1,1.587063,0.266667,NaN
1,CN,WEY7,2021-05-26 22:20:00,-2.79,-2.33,14.2,2.663972,14.224622,NaN
2,CN,NGG6,2019-12-16 00:09:00,0.04,-4.91,9.3,3.255315,25.379988,NaN
3,CA,FFM2,2015-11-16 05:30:00,NaN,NaN,NaN,NaN,84.764893,NaN
4,JP,QYY1,2018-10-24 01:11:00,NaN,NaN,NaN,NaN,0.000000,NaN
...,...,...,...,...,...,...,...,...,...
220486,BR,TMW2,2017-04-04 15:53:00,NaN,NaN,NaN,NaN,58.233528,NaN
220487,CA,GRQ5,2016-08-22 16:22:00,NaN,NaN,NaN,NaN,43.925217,NaN
220488,JP,VYJ1,2022-07-10 14:53:00,-2.87,1.22,27.4,2.405268,25.632092,NaN
220489,CN,QQW1,2020-12-28 14:38:00,-2.65,-0.64,8.2,2.199039,0.000000,NaN


In [34]:
sub = pd.read_csv('sample_submission.csv')
sub['CI_HOUR'] = full_df['CI_HOUR_NEW']
sub.to_csv("SOTATEST_TR_TE.csv",index=False)
sub

,SAMPLE_ID,CI_HOUR
0,TEST_000000,0.266667
1,TEST_000001,14.224622
2,TEST_000002,25.379988
3,TEST_000003,84.764893
4,TEST_000004,0.000000
...,...,...
220486,TEST_220486,58.233528
220487,TEST_220487,43.925217
220488,TEST_220488,25.632092
220489,TEST_220489,0.000000


# Train

In [42]:
mapping_dict_train

{('ZAG4', -3.18, -1.61, 6.7, 2.629349642): (Timestamp('2020-01-17 04:02:00'),
  0.0),
 ('WHH4', 6.1, -2.84, 28.1, 4.01621746): (Timestamp('2021-03-05 18:36:00'),
  68.39138889),
 ('JWI3', 2.82, 0.25, 28.6, 2.255079469): (Timestamp('2022-10-07 10:06:00'),
  0.0),
 ('UVK6', -2.16, 1.5, 25.6, 2.146870114): (Timestamp('2021-09-23 02:48:00'),
  2.721944444),
 ('TDA5', -4.88, 1.55, 21.9, 3.347518051): (Timestamp('2021-05-12 21:35:00'),
  7.374722222),
 ('QQW1', -0.52, 4.15, 10.5, 2.925160854): (Timestamp('2021-04-16 15:52:00'),
  0.0),
 ('NQO4', -1.25, 1.09, 20.3, 1.578840632): (Timestamp('2020-11-17 22:41:00'),
  18.00722222),
 ('VYJ1', 0.0, 0.0, 19.0, 0.0): (Timestamp('2019-06-09 23:32:00'),
  1.752777778),
 ('JEN5', -2.36, 5.42, 21.5, 3.684073668): (Timestamp('2022-05-24 04:49:00'),
  8.115277778),
 ('FCD5', 0.28, -11.42, -11.1, 5.715578805): (Timestamp('2022-01-12 09:50:00'),
  0.0),
 ('NGG6', -1.05, -2.26, 20.7, 2.071231677): (Timestamp('2020-09-23 20:11:00'),
  41.82277778),
 ('RKA2', 

In [39]:
import pandas as pd

# OLD-NEW 데이터 불러오기
old_new_df = test.copy()

# train 데이터 불러오기
train_df = pd.read_csv("train.csv")
train_df['ATA'] = pd.to_datetime(train_df['ATA'])

# 매핑 딕셔너리 생성 (train 데이터를 기반으로)
mapping_dict_train = {
    (row['ARI_PO'], row['U_WIND'], row['V_WIND'], row['AIR_TEMPERATURE'], row['BN']): (row['ATA'], row['CI_HOUR'])
    for _, row in train_df.iterrows() 
    if row['CI_HOUR'] <= 72 and not any(pd.isna([row['ARI_PO'], row['U_WIND'], row['V_WIND'], row['AIR_TEMPERATURE'], row['BN']]))
}
'''
# 새로운 타겟 값 계산 함수
def compute_new_target_train(row):
    key = (row['ARI_PO'], row['U_WIND'], row['V_WIND'], row['AIR_TEMPERATURE'], row['BN'])
    if key in mapping_dict_train and mapping_dict_train[key][0] > row['ATA']:
        time_difference = (mapping_dict_train[key][0] - row['ATA']).total_seconds() / 3600
        return time_difference
    return None
'''
# 새로운 타겟 값 계산 함수
def compute_new_target_train(row):
    key = (row['ARI_PO'], row['U_WIND'], row['V_WIND'], row['AIR_TEMPERATURE'], row['BN'])
    if key in mapping_dict_train and mapping_dict_train[key][0] > pd.to_datetime(row['ATA']):
        time_difference = (mapping_dict_train[key][0] - pd.to_datetime(row['ATA'])).total_seconds() / 3600
        return time_difference
    return None

# OLD-NEW 데이터프레임의 CI_HOUR_NEW 열 수정
old_new_df['New_Target'] = old_new_df.apply(compute_new_target_train, axis=1)

# 수정된 OLD-NEW 데이터프레임 저장
#old_new_df.to_csv("SOTATEST_TR.csv", index=False)
old_new_df

,ARI_CO,ARI_PO,ATA,U_WIND,V_WIND,AIR_TEMPERATURE,BN,CI_HOUR_NEW,New_Target
0,SG,GIW5,2020-06-18 11:58,0.37,1.63,27.1,1.587063,4.018523,NaN
1,CN,WEY7,2021-05-26 22:20,-2.79,-2.33,14.2,2.663972,14.224622,NaN
2,CN,NGG6,2019-12-16 0:09,0.04,-4.91,9.3,3.255315,25.379988,30.983333
3,CA,FFM2,2015-11-16 5:30,NaN,NaN,NaN,NaN,84.764893,NaN
4,JP,QYY1,2018-10-24 1:11,NaN,NaN,NaN,NaN,0.000000,NaN
...,...,...,...,...,...,...,...,...,...
220486,BR,TMW2,2017-04-04 15:53,NaN,NaN,NaN,NaN,58.233528,NaN
220487,CA,GRQ5,2016-08-22 16:22,NaN,NaN,NaN,NaN,43.925217,NaN
220488,JP,VYJ1,2022-07-10 14:53,-2.87,1.22,27.4,2.405268,25.632092,10.916667
220489,CN,QQW1,2020-12-28 14:38,-2.65,-0.64,8.2,2.199039,0.000000,NaN


In [41]:
import numpy as np
old_new_df['New_Target'] = np.where((old_new_df['New_Target'] > 2159) | old_new_df['New_Target'].isna(), np.nan, old_new_df['New_Target'])
old_new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220491 entries, 0 to 220490
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   ARI_CO           220491 non-null  object 
 1   ARI_PO           220491 non-null  object 
 2   ATA              220491 non-null  object 
 3   U_WIND           128766 non-null  float64
 4   V_WIND           128766 non-null  float64
 5   AIR_TEMPERATURE  128245 non-null  float64
 6   BN               128766 non-null  float64
 7   CI_HOUR_NEW      220491 non-null  float64
 8   New_Target       34865 non-null   float64
dtypes: float64(6), object(3)
memory usage: 15.1+ MB


In [40]:
#28191 / 1hour
#30567 / 6hours
#32327 / 12
#34032 / 24hours
#34834 / 48
#34865 / 72
#34758 / 96
#33355   / 8640

In [19]:
old_new_df['CI_HOUR_NEW'] = old_new_df['CI_HOUR_NEW'].where(old_new_df['New_Target'].isna(), old_new_df['New_Target'])
old_new_df

,ARI_CO,ARI_PO,ATA,U_WIND,V_WIND,AIR_TEMPERATURE,BN,CI_HOUR_NEW,New_Target
0,SG,GIW5,2020-06-18 11:58,0.37,1.63,27.1,1.587063,0.266667,0.266667
1,CN,WEY7,2021-05-26 22:20,-2.79,-2.33,14.2,2.663972,14.224622,NaN
2,CN,NGG6,2019-12-16 0:09,0.04,-4.91,9.3,3.255315,30.983333,30.983333
3,CA,FFM2,2015-11-16 5:30,NaN,NaN,NaN,NaN,84.764893,NaN
4,JP,QYY1,2018-10-24 1:11,NaN,NaN,NaN,NaN,0.000000,NaN
...,...,...,...,...,...,...,...,...,...
220486,BR,TMW2,2017-04-04 15:53,NaN,NaN,NaN,NaN,58.233528,NaN
220487,CA,GRQ5,2016-08-22 16:22,NaN,NaN,NaN,NaN,43.925217,NaN
220488,JP,VYJ1,2022-07-10 14:53,-2.87,1.22,27.4,2.405268,10.916667,10.916667
220489,CN,QQW1,2020-12-28 14:38,-2.65,-0.64,8.2,2.199039,0.000000,NaN


In [15]:
sub = pd.read_csv('sample_submission.csv')
sub['CI_HOUR'] = old_new_df['CI_HOUR_NEW']
sub.to_csv("SOTATEST_TR_PO_CHANGE_CO_24to8640.csv",index=False)
sub

,SAMPLE_ID,CI_HOUR
0,TEST_000000,4.018523
1,TEST_000001,14.224622
2,TEST_000002,30.983333
3,TEST_000003,84.764893
4,TEST_000004,0.000000
...,...,...
220486,TEST_220486,58.233528
220487,TEST_220487,43.925217
220488,TEST_220488,10.916667
220489,TEST_220489,0.000000


In [28]:
old_new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220491 entries, 0 to 220490
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   ARI_CO           220491 non-null  object 
 1   ARI_PO           220491 non-null  object 
 2   ATA              220491 non-null  object 
 3   U_WIND           128766 non-null  float64
 4   V_WIND           128766 non-null  float64
 5   AIR_TEMPERATURE  128245 non-null  float64
 6   BN               128766 non-null  float64
 7   CI_HOUR_NEW      220491 non-null  float64
 8   New_Target       30772 non-null   float64
dtypes: float64(6), object(3)
memory usage: 15.1+ MB
